In [23]:
import random
import pyzx as zx
from pyzx.web import compute_pauli_webs, compute_detecting_regions, compute_stabilisers
from pyzx import VertexType

This notebook demonstrates the use of the `compute_pauli_web` function from `pyzx.web`. Given a ZX diagram, the function returns two lists of Pauli webs: `stabilisers, regions` consisting of a generating set of all detecting regions and a generating a set of stabilising webs that forms a basis for the diagram's stabilisers when restricted to its boundary.
Together, `stabilisers` and `regions` generate all Pauli webs on the diagram.

In contrast to the version provided by `pyzx.pauliweb`, this function also works for non-unitary circuits. However, it does not provide a temporal ordering of the vertices, required when thinking about Pauli webs in Clifford+T circuits.

In [ ]:
# Example of computing Pauli webs for a single CNOT
c = zx.qasm("""
qreg q[2];
cx q[0], q[1];
""")
g = c.to_graph()
stabs, regions = compute_pauli_webs(g)
zx.draw(g, labels=True)

In [25]:
# There are 4 Pauli webs corresponding to the two outputs
for stab in stabs:
    zx.draw(g, labels=True, pauli_web=stab)

In [29]:
# A more complicated example with preparations and measurements
random.seed(1339)

no_qubits = 7
main = zx.generate.cliffords(qubits=no_qubits, depth=25)

preps = zx.Graph()
measurement = zx.Graph()
for q in range(no_qubits):
    p_1 = preps.add_vertex(random.sample([VertexType.Z, VertexType.X, VertexType.BOUNDARY], 1)[0], qubit=q, row=0)
    p_2 = preps.add_vertex(VertexType.BOUNDARY, qubit=q, row=1)
    preps.add_edge((p_1, p_2))
    
    m_1 = measurement.add_vertex(VertexType.BOUNDARY, qubit=q, row=0)
    m_2 = measurement.add_vertex(random.sample([VertexType.Z, VertexType.X, VertexType.BOUNDARY], 1)[0], qubit=q, row=1)
    measurement.add_edge((m_1, m_2))


preps.auto_detect_io()
measurement.auto_detect_io()
main.auto_detect_io()
g2 = preps + main + measurement
zx.draw(g2, labels=True)

stabs, detect = compute_pauli_webs(g2)
zx.draw(g2, pauli_web=stabs[0])
zx.draw(g2, pauli_web=detect[0])

To speed up computation, when only interested in stabilising Pauli webs or detecting regions, one can use the respective functions `compute_detecting_regions` and `compute_stabilisers`.

In [27]:
stabs2 = compute_stabilisers(g2)
detect2 = compute_detecting_regions(g2)

assert(len(stabs) == len(stabs2))
assert(len(detect) == len(detect2))

zx.draw(g2, pauli_web=stabs2[0])
